In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from bs4 import BeautifulSoup
import gzip
from pathlib import Path
from datetime import datetime
import re
from glob import glob
import pandas as pd
from time import sleep
from random import uniform

In [2]:
url = 'https://dunksandthrees.com/'
chromedriver_path = Path().joinpath(
    '/Users','johnschroeder','Documents','gambling_stuff','chromedriver')
base_path = '/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes'

In [3]:
chrome_options = ChromeOptions()
# chrome_options.add_argument('--headless')

In [4]:
driver = Chrome(executable_path=chromedriver_path, options=chrome_options)

In [5]:
response = driver.get(url)

In [14]:
soup = BeautifulSoup(driver.page_source)
raw_page_date = soup.find('span', {'class': 'page-date'}).text
print(raw_page_date)
ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')
raw_file = Path(base_path).joinpath(*ymd, 'dunksandthrees.html')
team_stats = soup.find('div', {'id': 'team-stats'})
Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
with open(raw_file, 'w') as fp:
    fp.write(str(team_stats.prettify()))

June  1, 2021


In [19]:
# iterate through multiple clicks to pull multiple raw files
nbr_clicks = 5

for click in range(nbr_clicks):
    soup = BeautifulSoup(driver.page_source)
    raw_page_date = soup.find('span', {'class': 'page-date'}).text
    print(raw_page_date)
    ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')
    raw_file = Path(base_path).joinpath(*ymd).with_suffix('.html')
    team_stats = soup.find('div', {'id': 'team-stats'})
    Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
    with open(raw_file, 'w') as fp:
        fp.write(str(team_stats.prettify()))
    driver.find_element_by_class_name('date-button').click()
    sleep(uniform(1.5, 2.5))

May 12, 2021
May 11, 2021
May 10, 2021
May  9, 2021
May  8, 2021


In [115]:
# collect all raw files and combine to a single dataframe

In [20]:
curated_file = '/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv'

In [21]:
stats_headers = team_stats.find('tr', {'class': 'main-header-tr'})

In [22]:
df_headers = [re.sub('\W+', '', _.text).lower() for _ in stats_headers.find_all('th')]

In [23]:
df_headers

['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl']

In [24]:
df_headers1 = ['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl',
 'date']

In [25]:
raw_files = glob('/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings/dunks_and_threes/*/*/*.html')
df_data = []
for raw_file in raw_files:
    with open(raw_file, 'r') as fp:
        raw_html = fp.read()
    team_stats = BeautifulSoup(raw_html)
    team_rows = team_stats.find_all('tr', {'class': 'stat-row'})
    replaces = re.compile(r'(\+)|(\s+)')
    file_date = '-'.join(Path(raw_file).with_suffix('').parts[-3:])
    for team_row in team_rows:
        row_data = [re.sub(replaces, '', _.text).strip() for _ in team_row.find_all('td')]
        row_data.append(file_date)
        df_data.append(row_data)

In [26]:
incr_df = pd.DataFrame(df_data, columns=df_headers1)

In [27]:
incr_df['team'].unique()

array(['1HOU', '2GSW', '4OKC', '2BOS', '1TOR', '3POR', '8WAS', '11NYK',
       '5IND', '9DET', '7SAS', '5UTA', '9DEN', '3PHI', '14ORL', '8MIN',
       '10CHA', '6NOP', '10LAC', '4CLE', '14MEM', '11LAL', '7MIL',
       '12BKN', '6MIA', '15ATL', '13DAL', '15PHX', '13CHI', '12SAC',
       '1MIL', '1GSW', '2DEN', '2TOR', '4BOS', '6OKC', '4HOU', '8LAC',
       '13NOP', '8DET', '10MIA', '11MIN', '9SAC', '7ORL', '6BKN', '14DAL',
       '9CHA', '10LAL', '12MEM', '11WAS', '12ATL', '15NYK', '14CLE',
       '1UTA', '3MIL', '2PHX', '3LAC', '4DEN', '1PHI', '7LAL', '2BKN',
       '5ATL', '5DAL', '9MEM', '7BOS', '11NOP', '4NYK', '10IND', '10SAS',
       '12TOR', '8GSW', '11CHI', '6POR', '8CHA', '15DET', '9WAS', '13OKC',
       '15HOU', '14MIN', '13ORL', '2PHI', '1BKN', '5LAL', '6DAL', '6BOS',
       '8MEM', '7POR', '10GSW', '9IND', '7MIA', '9SAS', '10WAS', '13CLE',
       '2UTA', '1PHX', '4LAC', '3DEN', '11TOR', '9GSW', '12CHI', '13MIN',
       '14OKC', '1LAL', '2LAC', '3BOS', '7DAL', '6UTA', '5OKC',

In [28]:
incr_df['team'] = incr_df['team'].replace(regex='\d+', value='')

In [124]:
incr_df = pd.DataFrame(df_data, columns=df_headers)
incr_df['date'] = str(datetime.strptime(raw_page_date, '%B %d, %Y').date())

In [29]:
incr_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,HOU,14-4,10.21,114.51,104.39,0.38,0.16,0.29,99.08,14.18,15.027,2017-11-23
1,GSW,13-5,9.52,113.82,104.38,0.44,0.23,0.28,102.64,13.72,14.48,2017-11-23
2,OKC,8-9,5.73,106.715,101.03,0.111,-0.222,0.37,96.920,14.310,15.430,2017-11-23
3,BOS,16-3,5.64,105.618,100.01,-1.829,-0.120,-1.730,96.524,15.021,14.822,2017-11-23
4,TOR,11-6,5.55,111.54,106.013,-0.526,-0.227,-0.324,98.215,14.716,14.718,2017-11-23


In [125]:
current_curated_df = pd.read_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv')

In [126]:
current_curated_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,OKC,21-48,-10.330,103.230,113.522,0.510,0.111,0.410,101.370,14.312,14.120,2021-05-09
1,GSW,35-33,0.515,110.223,109.750,-0.116,-0.125,0.080,102.520,13.630,14.517,2021-05-09
2,LAC,45-23,6.120,117.140,111.090,-0.114,-0.020,-0.017,97.427,15.026,14.618,2021-05-09
3,PHI,47-21,5.630,113.113,107.520,-0.012,-0.019,-0.016,99.812,14.716,14.110,2021-05-09
4,PHX,48-20,5.640,116.570,111.080,-0.122,0.180,-0.227,97.426,14.822,14.727,2021-05-09


In [127]:
incr_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,UTA,50-19,8.91,117.15,108.23,-0.013,-0.018,-0.014,99.115,14.719,14.37,2021-05-10
1,LAC,45-23,6.12,117.14,111.09,-0.115,-0.021,-0.016,97.427,15.026,14.618,2021-05-10
2,PHI,47-21,5.63,113.113,107.52,-0.014,-0.020,-0.015,99.812,14.716,14.11,2021-05-10
3,PHX,48-20,5.64,116.67,111.08,-0.122,0.18,-0.327,97.426,14.822,14.727,2021-05-10
4,MIL,43-25,5.15,116.86,111.710,-0.329,-0.022,-0.328,102.33,13.62,14.619,2021-05-10


In [128]:
revised_df = pd.concat([current_curated_df, incr_df]).sort_values('date', ascending=False)

In [30]:
incr_df.sort_values('date', ascending=False).to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv', index=False)

In [129]:
revised_df.to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv', index=False)